In [0]:
%sql
SHOW VOLUMES IN workspace.default;


database,volume_name


In [0]:

dbutils.widgets.dropdown(
    "layer",
    "bronze",
    ["bronze", "silver", "gold"]
)

layer = dbutils.widgets.get("layer")

from pyspark.sql import functions as F

CATALOG = "workspace"
SCHEMA  = "default"

BRONZE_TABLE = f"{CATALOG}.{SCHEMA}.bronze_events"
SILVER_TABLE = f"{CATALOG}.{SCHEMA}.silver_events"
GOLD_TABLE   = f"{CATALOG}.{SCHEMA}.gold_event_summary"

def run_layer(layer_name):

    if layer_name == "bronze":
        data = [
            (1, 101, "click", 0),
            (2, 102, "purchase", 250),
            (3, 101, "click", 0),
            (4, 103, "purchase", 500),
            (5, 104, "click", 0)
        ]

        df = spark.createDataFrame(
            data,
            ["event_id", "user_id", "event_type", "amount"]
        )

        df.withColumn("ingestion_ts", F.current_timestamp()) \
          .write.format("delta") \
          .mode("overwrite") \
          .saveAsTable(BRONZE_TABLE)

        print("✅ Bronze table created")

    elif layer_name == "silver":
        spark.table(BRONZE_TABLE) \
            .dropDuplicates(["event_id"]) \
            .dropna() \
            .write.format("delta") \
            .mode("overwrite") \
            .saveAsTable(SILVER_TABLE)

        print("✅ Silver table created")

    elif layer_name == "gold":
        spark.table(SILVER_TABLE) \
            .groupBy("event_type") \
            .agg(
                F.count("*").alias("total_events"),
                F.sum("amount").alias("total_amount")
            ) \
            .write.format("delta") \
            .mode("overwrite") \
            .saveAsTable(GOLD_TABLE)

        print("✅ Gold table created")

run_layer(layer)


✅ Bronze table created
